In [1]:
#export TRANSFORMERS_CACHE=/mnt/sda/dongkeun/huggingface

import os
os.environ['TRANSFORMERS_CACHE'] = '/mnt/sda/dongkeun/huggingface'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM
import torch

DEVICE = 'cuda:0'

tokenizer = AutoTokenizer.from_pretrained('/home/dongkeun/axolotl/checkpoints/kogemma-hermes_en-orca-wiki-alpaca-lima')
model = AutoPeftModelForCausalLM.from_pretrained('/home/dongkeun/axolotl/checkpoints/kogemma-hermes_en-orca-wiki-alpaca-lima/checkpoint-380', torch_dtype=torch.bfloat16).to(DEVICE)

/mnt/nas/dongkeun/.conda/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/nas/dongkeun/.conda/envs/torch/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 6/6 [00:01<00:00,  3.58it/s]


In [6]:
model.push_to_hub('kogemma-hermes_en-orca-wiki-alpaca-lima', private=True)
tokenizer.push_to_hub('kogemma-hermes_en-orca-wiki-alpaca-lima', private=True)

adapter_model.safetensors: 100%|██████████| 200M/200M [00:25<00:00, 7.81MB/s] 

tokenizer.model: 100%|██████████| 4.24M/4.24M [00:01<00:00, 2.65MB/s]

tokenizer.json: 100%|██████████| 17.5M/17.5M [00:03<00:00, 5.34MB/s]

Upload 2 LFS files: 100%|██████████| 2/2 [00:03<00:00,  1.82s/it]


CommitInfo(commit_url='https://huggingface.co/DKYoon/kogemma-hermes_en-orca-wiki-alpaca-lima/commit/c32a698678d3d30a6538f29e1dc58e10511555f0', commit_message='Upload tokenizer', commit_description='', oid='c32a698678d3d30a6538f29e1dc58e10511555f0', pr_url=None, pr_revision=None, pr_num=None)

In [4]:
content = '''우리나라의 전통적인 음식은 무엇인가요?
'''

content = content.strip()

messages = [
    {"role": "user", "content": content},
]

encodeds = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(encodeds)
encodeds = tokenizer(encodeds, return_tensors='pt')['input_ids']
print(encodeds)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
우리나라의 전통적인 음식은 무엇인가요?<|im_end|>
<|im_start|>assistant

tensor([[     2, 235322, 235371,    571, 235298,   2997,  73786,   9020,    108,
           2045,    708,    476,  10055,  20409,  35606, 235371,    571, 235298,
            615,  73786,    108, 235322, 235371,    571, 235298,   2997,  73786,
           1645,    108, 237312, 236432, 236915, 236950, 236137,  31087, 238693,
          85024, 107342, 238186, 236648,  60331, 242513, 236589, 236361, 237526,
         181537, 235371,    571, 235298,    615,  73786,    108, 235322, 235371,
            571, 235298,   2997,  73786, 105776,    108]])


In [5]:
model_inputs = encodeds.to(DEVICE)
generated_ids = model.generate(model_inputs, max_new_tokens=256, do_sample=False, eos_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<bos><|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
우리나라의 전통적인 음식은 무엇인가요?<|im_end|>
<|im_start|>assistant
우리나라의 전통적인 음식은 김치, 불고기, 비빔밥 등이 있습니다. 김치는 배추, 무, 오이 등의 채소를 소금에 절여 고춧가루, 마늘, 생강 등의 양념을 버무려 만든 음식으로, 한국인의 식탁에서 빠지지 않는 반찬입니다. 불고기는 소고기를 얇게 저며 양념에 재워 구워 먹는 음식으로, 한국인들이 좋아하는 음식 중 하나입니다. 비빔밥은 여러 가지 나물과 고기를 넣고 고추장이나 간장으로 비벼 먹는 음식으로, 한국인들이 좋아하는 음식 중 하나입니다.<|im_end|>
<eos>


In [11]:
content = '''
두개의 테이블 “주문”과 “고객” 가진 데이터베이스가 있습니다. “주문” 테이블에는 “주문_id” (정수), “고객_id” (정수), “주문_날짜” (날짜), “금액” (소수), “상태” (문자) 칼럼이 있고, “고객”  테이블에는 “고객_id” (정수), “이름” (문자), “이메일” (문자), “멤버쉽_상태” (문자) 칼럼이 있습니다.

각 고객이 “완료”된 주문에 대해 지불한 총 금액을 반환하는 SQL 쿼리를 작성하세요. “활성화” 된 멤버쉽 상태를 가지며 지불한 총 금액이 1000이 넘는 고객에 한해 고객 이름, 이메일, 총 금액을 반환해야합니다. 결과를 지불한 총 금액에 대해 내림차순으로 정렬하세요.'''

content = content.strip()

messages = [
    {"role": "user", "content": content},
]

encodeds = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(encodeds)
encodeds = tokenizer(encodeds, return_tensors='pt')['input_ids']
print(encodeds)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
두개의 테이블 “주문”과 “고객” 가진 데이터베이스가 있습니다. “주문” 테이블에는 “주문_id” (정수), “고객_id” (정수), “주문_날짜” (날짜), “금액” (소수), “상태” (문자) 칼럼이 있고, “고객”  테이블에는 “고객_id” (정수), “이름” (문자), “이메일” (문자), “멤버쉽_상태” (문자) 칼럼이 있습니다.

각 고객이 “완료”된 주문에 대해 지불한 총 금액을 반환하는 SQL 쿼리를 작성하세요. “활성화” 된 멤버쉽 상태를 가지며 지불한 총 금액이 1000이 넘는 고객에 한해 고객 이름, 이메일, 총 금액을 반환해야합니다. 결과를 지불한 총 금액에 대해 내림차순으로 정렬하세요.<|im_end|>
<|im_start|>assistant

tensor([[     2, 235322, 235371,    571, 235298,   2997,  73786,   9020,    108,
           2045,    708,    476,  10055,  20409,  35606, 235371,    571, 235298,
            615,  73786,    108, 235322, 235371,    571, 235298,   2997,  73786,
           1645,    108, 238928, 237936, 236137, 108214, 210938,   1080, 237014,
         237465, 235369, 237233,   1080, 236464, 240526, 235369,  23248, 237589,
         112762, 239571, 131006, 236361,  55496, 235265,   1080, 237014, 237465,
         235369, 108214, 210938,  96564,   1080, 237014, 2374

In [12]:
model_inputs = encodeds.to(DEVICE)
generated_ids = model.generate(model_inputs, max_new_tokens=256, do_sample=False, eos_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<bos><|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
두개의 테이블 “주문”과 “고객” 가진 데이터베이스가 있습니다. “주문” 테이블에는 “주문_id” (정수), “고객_id” (정수), “주문_날짜” (날짜), “금액” (소수), “상태” (문자) 칼럼이 있고, “고객”  테이블에는 “고객_id” (정수), “이름” (문자), “이메일” (문자), “멤버쉽_상태” (문자) 칼럼이 있습니다.

각 고객이 “완료”된 주문에 대해 지불한 총 금액을 반환하는 SQL 쿼리를 작성하세요. “활성화” 된 멤버쉽 상태를 가지며 지불한 총 금액이 1000이 넘는 고객에 한해 고객 이름, 이메일, 총 금액을 반환해야합니다. 결과를 지불한 총 금액에 대해 내림차순으로 정렬하세요.<|im_end|>
<|im_start|>assistant
```sql
SELECT c.name, c.email, SUM(o.amount) AS total_amount
FROM customer c
JOIN order o ON c.customer_id = o.customer_id
WHERE o.status = 'completed'
  AND c.membership_status = 'active'
  AND SUM(o.amount) > 1000
GROUP BY c.name, c.email
ORDER BY total_amount DESC;
```<|im_end|>
<eos>


In [14]:
content = '''
과일 가게에는 복숭아, 망고, 그리고 사과, 총 3개의 과일을 팔고 있습니다. 복숭아는 사과보다 비쌉니다. 망고가 가장 저렴합니다.
선택지:
(A) 복숭아가 두번째로 비쌉니다. (B) 망고가 두번째로 비쌉니다. (C) 사과가 두번째로 비쌉니다.

정답을 말하기 전, 생각을 단계별로 차근차근 먼저 말하고 마지막에 정답을 말하세요.
'''

content = content.strip()

messages = [
    {"role": "system", "content": "You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps."},
    {"role": "user", "content": content},
]

encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
encodeds = tokenizer(encodeds, return_tensors='pt')['input_ids']
print(encodeds)
model_inputs = encodeds.to(DEVICE)
generated_ids = model.generate(model_inputs, max_new_tokens=256, do_sample=False, eos_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

tensor([[     2, 235322, 235371,    571, 235298,   2997,  73786,   9020,    108,
           2045,    708,    671,  16481,  20409, 235265,   4926,    877,    692,
           2734,    692,    476,   6911, 235265,   3883,   6789,    603,    577,
           3407,    573,   6911,    685,  64214,    685,    692,    798, 235265,
           7859,  19330,    573,   6911,   1742,   4065, 235290,   1547, 235290,
           8847,    578,  22685,    861,   7161,  35606, 235371,    571, 235298,
            615,  73786,    108, 235322, 235371,    571, 235298,   2997,  73786,
           1645,    108, 237233, 236666,  23248, 237458,  96564, 122447, 244776,
         236655, 235269, 235248, 240547, 236464, 235269, 213512,  17309, 237233,
         235269, 168664, 235248, 235304, 237936, 236137,  65084, 236666, 236392,
         235248, 241634, 236464,  55496, 235265, 122447, 244776, 236655, 236214,
          17309, 237233, 180677,  51732, 253980,  12957, 235265, 235248, 240547,
         236464, 236361, 126

In [6]:
content = '''
'employees'라는 테이블이 있고, 이 테이블에는 'id' (정수), 'name' (텍스트), 'position' (텍스트), 'salary' (정수)라는 컬럼이 있습니다. 연봉이 50000 이상이며 직책이 'Manager'인 직원의 이름과 연봉을 찾는 SQL 쿼리를 작성하세요.
'''

content = content.strip()

messages = [
    {"role": "system", "content": "You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps."},
    {"role": "user", "content": content},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors='pt', add_generation_prompt=True)
model_inputs = encodeds.to(DEVICE)
generated_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=False, eos_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<|im_start|>system
You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.<|im_end|>
<|im_start|>user
'employees'라는 테이블이 있고, 이 테이블에는 'id' (정수), 'name' (텍스트), 'position' (텍스트), 'salary' (정수)라는 컬럼이 있습니다. 연봉이 50000 이상이며 직책이 'Manager'인 직원의 이름과 연봉을 찾는 SQL 쿼리를 작성하세요.<|im_end|>
<|im_start|>assistant
연봉이 50000 이상이고 직책이 'Manager'인 직원의 이름과 연봉을 찾는 SQL 쿼리를 작성하기 위해 다음 단계를 수행합니다:

1. 'employees' 테이블에서 'position' 컬럼에 'Manager'라는 값이 있는 행을 식별합니다.
2. 'position' 컬럼에서 'Manager'라는 값이 있는 행을 식별한 후, 'salary' 컬럼에서 50000 이상의 값이 있는 행을 필터링합니다.
3. 'salary' 컬럼에서 50000 이상의 값이 있는 행을 식별한 후, 'name' 컬럼에서 해당 행의 이름을 검색합니다.

SQL 쿼리는 다음과 같습니다:

SELECT name, salary
FROM employees
WHERE position = 'Manager'
AND salary > 50000;

이 쿼리는 'position' 컬럼에서 'Manager'라는 값이 있는 행을 필터링하고 'salary' 컬럼에서 50000 이상의 값이 있는 행을 필터링합니다. 그런 다음 'name' 컬럼에서 해당 행의 이름을 검색합니다.<|im_end|>
<eos>
